## Working with rHEALpix in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/04_rhealpix.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/04_rhealpix.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/04_rhealpix.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/04_rhealpix.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/04_rhealpix.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2rhealpix

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2rhealpix

lat = 10.775276
lon = 106.706797
res = 12
rhealpix_id = latlon2rhealpix(lat, lon, res)
rhealpix_id

### rHEALPix to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.rhealpix2geo import rhealpix2geo

rhealpix_geo = rhealpix2geo(rhealpix_id)
rhealpix_geo

### rHEALPix to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.rhealpix2geo import rhealpix2geojson

rhealpix_geojson = rhealpix2geojson(rhealpix_id)
rhealpix_geojson

### Vector to rHEALPix

In [ ]:
from vgrid.conversion.vector2dggs.vector2rhealpix import vector2rhealpix

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_rhealpix = vector2rhealpix(file_path, resolution=10, compact=False, 
                                    predicate = "intersects",output_format="gpd")
# Visualize the output 
vector_to_rhealpix.plot(edgecolor="white")

### rHEALPix Compact

In [ ]:
from vgrid.conversion.dggscompact.rhealpixcompact import rhealpixcompact

rhealpix_compacted = rhealpixcompact(
    vector_to_rhealpix, rhealpix_id="rhealpix", output_format="gpd")
rhealpix_compacted.plot(edgecolor="white")


### rHEALPix Expand

In [ ]:
from vgrid.conversion.dggscompact.rhealpixcompact import rhealpixexpand

rhealpix_expanded = rhealpixexpand(
    vector_to_rhealpix, resolution=11, output_format="gpd"
)
rhealpix_expanded.plot(edgecolor="white")

### rHEALPix Binning

In [ ]:
from vgrid.binning.rhealpixbin import rhealpixbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
rhealpix_bin = rhealpixbin(file_path, resolution=10, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
rhealpix_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)   

### Raster to rHEALPix

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2rhealpix import raster2rhealpix
from vgrid.utils.io import download_file          

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_rhealpix =  raster2rhealpix(raster_file,output_format="gpd")
# Visualize the output 
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

rhealpix_layer = folium.GeoJson(
    raster_to_rhealpix,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["rhealpix", "resolution", "band_1", "band_2", "band_3", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Band 1", "Band 2", "Band 3", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpix_layer.get_bounds())

# Display the map
m

### rHEALPix Generator

In [ ]:
from vgrid.generator.rhealpixgrid import rhealpixgrid

rhealpix_grid = rhealpixgrid(resolution=1,output_format="gpd")
# rhealpix_grid = rhealpixgrid(resolution=10,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
rhealpix_grid.plot(edgecolor="white")

### rHEALPix Inspect

In [ ]:
from vgrid.stats.rhealpixstats import rhealpixinspect
resolution = 4
rhealpix_inspect = rhealpixinspect(resolution)
rhealpix_inspect.head()

### Distribution of rHEALPix Area Distortions

In [ ]:
from vgrid.stats.rhealpixstats import rhealpix_norm_area
rhealpix_norm_area(rhealpix_inspect)

### Distribution of rHEALPix IPQ Compactness

In [ ]:
from vgrid.stats.rhealpixstats import rhealpix_compactness
rhealpix_compactness(rhealpix_inspect)

### rHEALPix Statistics

In [ ]:
from vgrid.stats.rhealpixstats import rhealpixstats
rhealpixstats('km')       